# ADNI and the K-operator

Using the visualization of connectivity matrices mutuated from Sidney Hough, Julian Quevedo, and Pino Cholsaipant (https://i.imgur.com/FYaEgKW.jpg).

Here: application to the ADNI dataset (Alzheimer-Perusini) disease.

Here: Harvard-Oxford atlas, and focus on the subcortical, hippocampal, and temporal regions

All the imports and installs we'll need.


In [ ]:
import os
import torch
from torch import nn
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

from nilearn import datasets
from nilearn import plotting
from nilearn.input_data import NiftiMapsMasker
from nilearn.connectome import ConnectivityMeasure

import torch.nn.functional as F
from torch.nn import Sequential, Linear, ReLU, GRU, BatchNorm1d
from torch_geometric.nn import EdgeConv, GCNConv, GraphConv
from torch_geometric.nn import global_mean_pool
from torch_geometric.data import InMemoryDataset, Data, DataLoader
from torch_geometric.utils import from_networkx

import networkx as nx
from networkx.convert_matrix import from_numpy_matrix

import os
import torch
from torch import nn
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

from nilearn import datasets
from nilearn import plotting
from nilearn.input_data import NiftiMapsMasker
from nilearn.connectome import ConnectivityMeasure

import torch.nn.functional as F
from torch.nn import Sequential, Linear, ReLU, GRU, BatchNorm1d
from torch_geometric.nn import EdgeConv, GCNConv, GraphConv
from torch_geometric.nn import global_mean_pool
from torch_geometric.data import InMemoryDataset, Data, DataLoader
from torch_geometric.utils import from_networkx

import networkx as nx
from networkx.convert_matrix import from_numpy_matrix

In [ ]:
from nilearn import datasets

# Fetch the Harvard-Oxford atlas
atlas = datasets.fetch_atlas_harvard_oxford('cort-maxprob-thr25-2mm')


#datasets.fetch_atlas_harvard_oxford('sub-maxprob-thr25-2mm')

# Print the labels and their corresponding indices
for idx, label in enumerate(atlas.labels):
    print(f"Index {idx}: {label}")


In [ ]:
# MALE patient 1

In [ ]:
######


# ADNI

####

# PART 1.

# loading DICOM image
# conversion to NIfTI
# analysis of connectivity with Nilearn

import os
import dicom2nifti
import dicom2nifti.settings as settings
import pydicom


dicom_folder = "/Users/mariamannone/Downloads/ADNI 4/002_S_5018/Resting_State_fMRI/2013-02-11_07_46_12.0/I358614"

output_folder = "/Users/mariamannone/Desktop/new_phd_/DICOM_converted_ADNI_6_AAL/"


# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Convert DICOM to NIfTI using dicom2nifti
settings.disable_validate_slice_increment()
dicom_files = [os.path.join(dicom_folder, file) for file in os.listdir(dicom_folder) if file.endswith(".dcm")]

# Load the DICOM data
dicom_data = [pydicom.dcmread(file) for file in dicom_files]

# Convert DICOM to NIfTI
nifti_file = os.path.join(output_folder, "output.nii.gz")
dicom2nifti.convert_directory(dicom_folder, nifti_file, compression=True, reorient=True)


In [ ]:
######

# PART 2.

# Opening the .nii file, and computing its connectivity etc. with Nilearn


from nilearn import datasets, input_data, connectome
import nibabel as nib
import matplotlib.pyplot as plt
import numpy as np


nifti_file = "/Users/mariamannone/Desktop/new_phd_/DICOM_converted_ADNI_6_AAL/output.nii.gz/501_resting_state_fmri.nii.gz"
fmri_img = nib.load(nifti_file)

# Load a standard atlas (e.g., Harvard-Oxford atlas)
atlas = datasets.fetch_atlas_harvard_oxford('cort-maxprob-thr25-2mm')

#datasets.fetch_atlas_harvard_oxford('sub-maxprob-thr25-2mm')

# Create a masker using the atlas
masker = input_data.NiftiLabelsMasker(labels_img=atlas.maps,
                                      standardize=True,
                                      memory='nilearn_cache')

# Extract time series from ROIs
time_series = masker.fit_transform(fmri_img)

# Compute correlation matrix
correlation_matrix = connectome.ConnectivityMeasure(kind='correlation')
correlation_matrix.fit([time_series])

# Get the computed connectivity matrix
connectivity_matrix = correlation_matrix.mean_

# Create a heatmap of the connectivity matrix with a colorbar
plt.imshow(connectivity_matrix, cmap='viridis', interpolation='nearest')
plt.colorbar(label='Correlation')
plt.title('Connectivity Matrix')
plt.xlabel('ROIs')
plt.ylabel('ROIs')
plt.show()



In [ ]:
# STORE

male_BL = connectivity_matrix 

In [ ]:
# April 19, clear command


del time_series, correlation_matrix, connectivity_matrix


In [ ]:
# OTHER Measurement, same male patient 1

In [ ]:
######


# ADNI

####

# PART 1.

# loading DICOM image
# conversion to NIfTI
# analysis of connectivity with Nilearn

import os
import dicom2nifti
import dicom2nifti.settings as settings
import pydicom

dicom_folder = "/Users/mariamannone/Downloads/ADNI 5/006_S_4153/Resting_State_fMRI/2011-11-04_11_36_04.0/I264987"

output_folder = "/Users/mariamannone/Desktop/new_phd_/DICOM_converted_ADNI_10_oxford/"

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Convert DICOM to NIfTI using dicom2nifti
settings.disable_validate_slice_increment()
dicom_files = [os.path.join(dicom_folder, file) for file in os.listdir(dicom_folder) if file.endswith(".dcm")]

# Load the DICOM data
dicom_data = [pydicom.dcmread(file) for file in dicom_files]

# Convert DICOM to NIfTI
nifti_file = os.path.join(output_folder, "output.nii.gz")
dicom2nifti.convert_directory(dicom_folder, nifti_file, compression=True, reorient=True)


In [ ]:
######

# PART 2.

# Opening the .nii file, and computing its connectivity etc. with Nilearn


from nilearn import datasets, input_data, connectome
import nibabel as nib
import matplotlib.pyplot as plt
import numpy as np


# Load the NIfTI file
nifti_file = "/Users/mariamannone/Desktop/new_phd_/DICOM_converted_ADNI_10_oxford/output.nii.gz/801_resting_state_fmri.nii.gz"
fmri_img = nib.load(nifti_file)

# Load a standard atlas (e.g., Harvard-Oxford atlas)
atlas = datasets.fetch_atlas_harvard_oxford('cort-maxprob-thr25-2mm')


# Create a masker using the atlas
masker = input_data.NiftiLabelsMasker(labels_img=atlas.maps,
                                      standardize=True,
                                      memory='nilearn_cache')

# Extract time series from ROIs
time_series = masker.fit_transform(fmri_img)

# Compute correlation matrix
correlation_matrix_fu1 = connectome.ConnectivityMeasure(kind='correlation')
correlation_matrix_fu1.fit([time_series])

# Get the computed connectivity matrix
connectivity_matrix_fu1 = correlation_matrix_fu1.mean_

# Create a heatmap of the connectivity matrix with a colorbar
plt.imshow(connectivity_matrix_fu1, cmap='viridis', interpolation='nearest')
plt.colorbar(label='Correlation')
plt.title('Connectivity Matrix')
plt.xlabel('ROIs')
plt.ylabel('ROIs')
plt.show()



In [ ]:
# STORE

male_FU = connectivity_matrix_fu1


In [ ]:
# April 19, clear command



del correlation_matrix, connectivity_matrix


In [ ]:
# K for the male 1, directly for the subcortical detail



# Define the lobe mapping with only the specified blocks
lobe_mapping = {
    'Selection_area': [8, 9, 10, 11, 12, 13, 14, 15, 16, 34, 35, 37, 38, 39, 40, 46]
}

matrix1 = male_BL # connectivity_matrix
matrix2 = male_FU # connectivity_matrix_fu1

# Compute the inverse of matrix1
inverse_matrix1 = np.linalg.inv(matrix1)


# Define the lobe mapping with only the specified blocks
lobe_mapping = {
    'Selection_area': [8, 9, 10, 11, 12, 13, 14, 15, 16, 34, 35, 37, 38, 39, 40, 46]
}


# Function to visualize submatrix for a given lobe
def visualize_submatrix_for_lobe(lobe):
    lobe_indices = lobe_mapping[lobe]
    submatrix = np.zeros((len(lobe_indices), len(lobe_indices)))
    submatrix_labels = [atlas_labels[i] for i in lobe_indices]  # Get ROI labels

    for i, roi1 in enumerate(lobe_indices):
        for j, roi2 in enumerate(lobe_indices):
            submatrix[i, j] = matrix2[roi1, roi2] * inverse_matrix1[roi2, roi1]

    # Plot heatmap for the submatrix
    title = f"K for parahippocampus and fusiform"
    plt.figure(dpi=300)
    plotting.plot_matrix(submatrix, labels=submatrix_labels, colorbar=True, vmax=1, vmin=-1, title=title)
    plotting.show()

# Visualize submatrices for each lobe
for lobe in lobe_mapping:
    visualize_submatrix_for_lobe(lobe)







In [ ]:
# After having computed data for other patients, we can try a prediction, given the baseline of a new patient.

In [ ]:
####

# PREDICTION

####


# Using a regression:

import numpy as np
from sklearn.linear_model import LinearRegression

# Defined before: female_BL, female_FU, male_BL, male_FU, new_patient
# define measured_new_patient_FU, K_measured
# compute estimated_new_patient_FU, K_predicted 

Matrix1 = female_BL
Matrix2 = female_FU
Matrix3 = male_BL
Matrix4 = male_FU

# a new male patient at the baseline: 
Matrix5 = new_patient

# his real 1st FU:
Matrix6 = measured_new_patient_FU

# Calculate K matrices
K_1 = np.dot(female_FU, np.linalg.inv(female_BL))
K_2 = np.dot(male_FU, np.linalg.inv(male_BL))


# Prepare your training data
X_train = np.vstack((female_BL.flatten(), male_BL.flatten()))  # Flatten the matrices and stack them vertically
y_train = np.array([K_1.flatten(), K_2.flatten()])  # Flatten the K matrices and convert to array

# Define the linear regression model
model = LinearRegression()

# Train the linear regression model
model.fit(X_train, y_train)

# Predict K_3 for the new patient
estimated_new_patient_FU = new_patient.flatten().reshape(1, 2304)
K_predicted = model.predict(estimated_new_patient_FU).reshape(1, 2304) 

# Reshape back to the original configuration
new_patient = new_patient.reshape(48, 48)
K_predicted = K_predicted.reshape(48, 48)




plt.imshow(K_predicted, cmap='viridis', interpolation='nearest')
plt.colorbar(label='Correlation')
plt.title('Predicted K')
plt.xlabel('ROIs')
plt.ylabel('ROIs')
plt.show()


K_measured = np.dot(Matrix6, np.linalg.inv(Matrix5))


plt.imshow(K_measured, cmap='viridis', interpolation='nearest')
plt.colorbar(label='Correlation')
plt.title('Measured K')
plt.xlabel('ROIs')
plt.ylabel('ROIs')
plt.show()


# Compute the Frobenius norm (distance) between the two matrices
frobenius_distance = np.linalg.norm(K_predicted - K_measured, ord='fro')

print("Frobenius Distance:", frobenius_distance)


# check the approximations

tolerance = 1 # e-6 # 10
is_close = np.allclose(K_predicted, K_measured, atol=tolerance)

if is_close:
    print("The predicted K-operator IS CLOSE to the real one, with a tolerance of ", tolerance)
else:
    print("The predicted K-operator IS NOT CLOSE to the real one, with a tolerance of ", tolerance)
    

In [ ]:
def visualize_submatrix_for_lobe(lobe):
    lobe_indices = lobe_mapping[lobe]
    submatrix = np.zeros((len(lobe_indices), len(lobe_indices)))
    submatrix_labels = [atlas_labels[i] for i in lobe_indices]  # Get ROI labels

    for i, roi1 in enumerate(lobe_indices):
        for j, roi2 in enumerate(lobe_indices):
            submatrix[i, j] = K_predicted[roi1, roi2] * inverse_matrix1[roi2, roi1]

    # Plot heatmap for the submatrix
    title = f"K for parahippocampus and fusiform"
    plt.figure(dpi=300)
    plotting.plot_matrix(submatrix, labels=submatrix_labels, colorbar=True, vmax=1, vmin=-1, title=title)
    plotting.show()

# Visualize submatrices for each lobe
for lobe in lobe_mapping:
    visualize_submatrix_for_lobe(lobe)
    
# Visualizing predicted K for the considered selection

In [ ]:
def visualize_submatrix_for_lobe(lobe):
    lobe_indices = lobe_mapping[lobe]
    submatrix = np.zeros((len(lobe_indices), len(lobe_indices)))
    submatrix_labels = [atlas_labels[i] for i in lobe_indices]  # Get ROI labels

    for i, roi1 in enumerate(lobe_indices):
        for j, roi2 in enumerate(lobe_indices):
            submatrix[i, j] = K_measured[roi1, roi2] * inverse_matrix1[roi2, roi1]

    # Plot heatmap for the submatrix
    title = f"K for parahippocampus and fusiform"
    plt.figure(dpi=300)
    plotting.plot_matrix(submatrix, labels=submatrix_labels, colorbar=True, vmax=1, vmin=-1, title=title)
    plotting.show()

# Visualize submatrices for each lobe
for lobe in lobe_mapping:
    visualize_submatrix_for_lobe(lobe)
    
# Visualizing predicted K for the considered selection